In [5]:
from transformers import BertTokenizerFast
import json

In [6]:
tokenizer_fast = BertTokenizerFast.from_pretrained('./save_tokenizer/')
print(tokenizer_fast)

PreTrainedTokenizerFast(name_or_path='./save_tokenizer/', vocab_size=21128, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [7]:
test_data = []

with open('result_data/ner.json', 'r', encoding='utf-8') as f:
    for ners_sentence in json.load(f):

        offset_mapping_output = tokenizer_fast(ners_sentence['sent'], return_offsets_mapping=True)["offset_mapping"]
        # 处理原句空格
        offset_mapping = []
        for i, (start, end) in enumerate(offset_mapping_output):
            if (end > 0) and (i >= 2):
                start -= (i - 1)
                end -= (i - 1)
            offset_mapping.append((start, end))

        start_mapping = {j[0]: i for i, j in enumerate(offset_mapping) if j != (0, 0)}
        # j[1] - 1表示该token结尾字符的位置
        end_mapping = {j[1] - 1: i for i, j in enumerate(offset_mapping) if j != (0, 0)}

        # example:{"[0, 2, '器官组织', '胸廓']": [1, 2], "[2, 4, '阴性表现', '对称']": [3, 4], xxxxxx}
        ner_idx_dict = dict()
        for ent in ners_sentence['ners']:
            start_idx, end_idx, entity_type, entity_text = ent[0], ent[1] - 1, ent[2], ent[3]
            if start_idx in start_mapping and end_idx in end_mapping:
                start_span = start_mapping[start_idx]
                end_span = end_mapping[end_idx]
            ner_idx_dict[str(ent)] = [start_span, end_span]

        # example:{"[0, 2, '器官组织', '胸廓']": [94, 95], "[2, 4, '阴性表现', '对称']": [96, 97], xxxxxx}
        ner_token_idx_dict = dict()
        # [CLS] 胸 廓 对 称 ， 气 管 居 中 。 所 见 骨 骼 骨 质 结 构 完 整 。 双 肺 纹 理 清 晰 。 两 肺 门 影 不 大 。 心 影 横 径 增 大 ， 左 心 缘 饱 满 。 两 侧 膈 面 光 整 ， 两 侧 肋 膈 角 锐 利 。 1 . 两 肺 未 见 明 显 活 动 性 病 变 ， 随 诊 。 2 . 心 影 改 变 请 结 合 临 床 。 [SEP] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [属性] [/属性] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阳性表现] [/阳性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [阴性表现] [/阴性表现] [器官组织] [/器官组织] [否定描述] [/否定描述] [修饰描述] [/修饰描述] [修饰描述] [/修饰描述] [异常现象] [/异常现象] [器官组织] [/器官组织] [异常现象] [/异常现象]
        sentence_text = '[CLS] ' + ners_sentence['sent'] + ' [SEP]'
        position_ids = [i for i in range(len(offset_mapping))]
        for ner in ners_sentence['ners']:
            sentence_text += f' [{ner[2]}] [/{ner[2]}]'  # 加入实体对应的类型
            ner_token_idx_dict[str(ner)] = [len(offset_mapping) + i for i in
                                            range(2)]  # 实体对应的类型(type)该token在sentence_text中的位置
            position_ids.extend(ner_idx_dict[str(ner)])  # 位置编码共享位置信息

        relation, relation_idx = [], []
        for sub in ners_sentence['ners']:  # sub:[0, 2, '器官组织', '胸廓']
            for obj in ners_sentence['ners']:
                if str(sub) == str(obj):
                    continue
                # relation,relation_idx可能为空列表
                relation.append(f'{str(sub)}\t{str(obj)}')
                relation_idx.append(ner_token_idx_dict[str(sub)] + ner_token_idx_dict[str(obj)])

        test_data.append({'sentence_text': sentence_text,
                          'position_ids': position_ids,
                          'relation': relation,
                          'relation_idx': relation_idx})

In [8]:
with open('result_data/ner_rel_predict.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)


